In [1]:
from italian_csv_type_prediction.dataframe_generators import SimpleDatasetGenerator
from italian_csv_type_prediction.models import TypePredictor
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from sklearn.metrics import accuracy_score, balanced_accuracy_score

In [2]:
def dataset_generation(number:int):
    return SimpleDatasetGenerator(verbose=False).build(number)

def _dataset_generation(args):
    return dataset_generation(*args)

def parallel_dataset_generation(number:int):
    processes = min(cpu_count(), number)
    with Pool(cpu_count()) as p:
        Xs, ys = list(zip(*tqdm(
            p.imap(_dataset_generation, (
                (number//processes, )
                for _ in range(processes)
            )),
            desc="Creating dataset",
            total=processes,
            leave=False
        )))
    return np.vstack(Xs), np.concatenate(ys)

In [3]:
x_train, y_train = parallel_dataset_generation(1000)
x_test, y_test = parallel_dataset_generation(1000)

In [4]:
model = TypePredictor()

model.fit(x_train, y_train)

In [5]:
y_pred = model._model.predict(x_test)
y_train_pred = model._model.predict(x_train)

In [6]:
accuracy_score(y_test, y_pred), balanced_accuracy_score(y_test, y_pred)

(0.9987905104949711, 0.9987691120985361)

In [7]:
accuracy_score(y_train, y_train_pred), balanced_accuracy_score(y_train, y_train_pred)

(0.9996664033008515, 0.999661182032328)

In [8]:
X, y = SimpleDatasetGenerator().generate_simple_dataframe(max_rows=10)

In [9]:
y

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
0,NaN,ItalianVAT,CadastreCode,Document,Tax,NaN,Address,ItalianZIPCode,NaN,Region,...,Country,NaN,Name,NaN,String,EMail,NaN,NaN,BiologicalSex,Boolean
1,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,NaN,Plate,Address,ItalianZIPCode,NaN,Region,...,Country,CountryCode,Name,Surname,NaN,NaN,PhoneNumber,Date,BiologicalSex,Boolean
2,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Error,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,Error,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
3,ItalianFiscalCode,NaN,CadastreCode,Document,Tax,Plate,NaN,NaN,ProvinceCode,Region,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,NaN
4,NaN,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,NaN,Region,...,Country,CountryCode,Name,Surname,String,EMail,NaN,Date,BiologicalSex,Boolean
5,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,NaN,Plate,Address,ItalianZIPCode,NaN,NaN,...,Country,CountryCode,Name,NaN,String,EMail,PhoneNumber,NaN,NaN,Boolean
6,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Tax,NaN,Address,Error,ProvinceCode,Region,...,Country,CountryCode,NaN,Surname,String,EMail,PhoneNumber,NaN,BiologicalSex,Boolean
7,NaN,Error,NaN,Document,Tax,NaN,Address,ItalianZIPCode,ProvinceCode,Region,...,NaN,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
8,ItalianFiscalCode,NaN,CadastreCode,Document,Tax,NaN,Address,ItalianZIPCode,ProvinceCode,Region,...,Country,CountryCode,Name,Surname,String,NaN,PhoneNumber,NaN,BiologicalSex,Boolean
9,NaN,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,NaN,ProvinceCode,Region,...,NaN,CountryCode,Name,Surname,String,EMail,NaN,Date,BiologicalSex,Error


In [10]:
model.predict_dataframe(X)

,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
0,NaN,ItalianVAT,CadastreCode,Document,Tax,NaN,Address,ItalianZIPCode,NaN,Region,...,Country,NaN,Name,NaN,String,EMail,NaN,NaN,BiologicalSex,Boolean
1,ItalianFiscalCode,ItalianVAT,CadastreCode,NaN,NaN,Plate,Address,ItalianZIPCode,NaN,Region,...,Country,CountryCode,Name,Surname,NaN,NaN,PhoneNumber,Date,BiologicalSex,Boolean
2,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Error,Plate,Address,ItalianZIPCode,ProvinceCode,Region,...,Error,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
3,ItalianFiscalCode,NaN,CadastreCode,Error,Tax,Plate,NaN,NaN,ProvinceCode,Region,...,Country,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,NaN
4,NaN,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,ItalianZIPCode,NaN,Region,...,Country,CountryCode,Name,Surname,String,EMail,NaN,Date,BiologicalSex,Boolean
5,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,NaN,Plate,Address,ItalianZIPCode,NaN,NaN,...,Country,CountryCode,Name,NaN,String,EMail,PhoneNumber,NaN,NaN,Boolean
6,ItalianFiscalCode,ItalianVAT,CadastreCode,Document,Tax,NaN,Address,Error,ProvinceCode,Region,...,Country,CountryCode,NaN,Surname,String,EMail,PhoneNumber,NaN,BiologicalSex,Boolean
7,NaN,Error,NaN,Document,Tax,NaN,Address,ItalianZIPCode,ProvinceCode,Region,...,NaN,CountryCode,Name,Surname,String,EMail,PhoneNumber,Date,BiologicalSex,Boolean
8,ItalianFiscalCode,NaN,CadastreCode,Document,Tax,NaN,Address,ItalianZIPCode,ProvinceCode,Region,...,Country,CountryCode,Name,Surname,String,NaN,PhoneNumber,NaN,BiologicalSex,Boolean
9,NaN,ItalianVAT,CadastreCode,Document,Tax,Plate,Address,NaN,ProvinceCode,Region,...,NaN,CountryCode,Name,Surname,String,EMail,NaN,Date,BiologicalSex,Error


In [11]:
from collections import Counter

mask = y_test != y_pred

true_labels = model._embedder._encoder.inverse_transform(y_test[mask])
predicted_labels = model._embedder._encoder.inverse_transform(y_pred[mask])

Counter(zip(true_labels, predicted_labels))

Counter({('Error', 'Document'): 174,
         ('NaN', 'CountryCode'): 114,
         ('CountryCode', 'NaN'): 79,
         ('Error', 'ProvinceCode'): 22,
         ('Year', 'Integer'): 3,
         ('Error', 'Integer'): 1,
         ('Error', 'Tax'): 10,
         ('String', 'Error'): 83,
         ('Integer', 'NaN'): 8,
         ('Error', 'Plate'): 2,
         ('Error', 'Float'): 1,
         ('ItalianZIPCode', 'Error'): 3,
         ('Error', 'CountryCode'): 31,
         ('NaN', 'Error'): 9,
         ('Year', 'Error'): 3,
         ('Document', 'Error'): 52,
         ('Error', 'Date'): 2,
         ('Error', 'ItalianFiscalCode'): 11,
         ('Error', 'ItalianVAT'): 11,
         ('Error', 'String'): 6,
         ('Name', 'Error'): 18,
         ('Error', 'Year'): 2,
         ('Error', 'Surname'): 4,
         ('Error', 'ItalianZIPCode'): 4,
         ('ItalianVAT', 'Error'): 10,
         ('ProvinceCode', 'CountryCode'): 39,
         ('String', 'PhoneNumber'): 3,
         ('ItalianFiscalCode', 'Err